#### Using UMich RSS data
https://ieee-dataport.org/open-access/crawdad-umichrss

In [3]:
import os
from numpy import linalg
from scipy import sparse
import math

import numpy as np
import pandas as pd

from math import sqrt, log
import statistics
import scipy.linalg
import copy

In [4]:
n_available_cores = len(os.sched_getaffinity(0))
print(n_available_cores)

14


In [5]:
data = []
with open("Z.txt") as f:
    for line in f:
        data.append([str(x) for x in line.split(sep='/n')])

In [6]:
a = []
for i in range(len(data)):
    a.append(list([float(x) for x in data[i][0].split(sep=',')]))
d = np.matrix(np.array(a))

In [ ]:
d

Применим разложение по сингулярным значениям (SVD), чтобы проверить, имеет ли центрированная матрица хорошую аппроксимацию низкого ранга.

In [8]:
main_arr = np.array(d)
u, s, vt = np.linalg.svd(main_arr)

In [9]:
s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2
    
s2matr=np.array(s2)    

total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1
print(round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

32.4 % singular values to capture 90 % variance


Введём аномалии, чтобы увидеть, как это влияет на результаты.

In [10]:

df = pd.DataFrame(d)
#используем экспоненциально-взвешенную скользящую среднюю (EWMA) для прогнозирования будущих записей на основе их прошлых значений
df_ewm = df.ewm(alpha=0.8, adjust=False).mean()
#используем максимальную разницу между фактическим и прогнозируемым значением в качестве размера вводимой аномалии
dt = df - df_ewm
max_by_columns = dt.max()
anomoly_size = max(max_by_columns)
print("Anomoly_size", anomoly_size)

#изменим долю записей для введения аномалий с 5% до 10%, а также масштабируем размер аномалии на s, который равен 0,5 или 1.
s_5 = anomoly_size*0.5
s_1 = anomoly_size*1

random_indx_y10 = np.random.randint(0, 181, size=round(182*3127*0.1))
random_indx_x10 = np.random.randint(0, 3126, size=round(182*3127*0.1))

Anomoly_size 14.753346168551886


In [11]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_5

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2
    
s2matr=np.array(s2)    
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 0.5, fraction = 10% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 0.5, fraction = 10%  64.3 % singular values to capture 90 % variance


In [12]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)//2):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_5

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2
    
s2matr=np.array(s2)    
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 0.5, fraction = 5% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 0.5, fraction = 5%  53.300000000000004 % singular values to capture 90 % variance


In [11]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_1

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2
    
s2matr=np.array(s2)    
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1
print("With s = 1, fraction = 10% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 1, fraction = 10%  78.60000000000001 % singular values to capture 90 % variance


In [12]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)//2):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_1

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2
    
s2matr=np.array(s2)    
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 1, fraction = 5% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 1, fraction = 5%  73.6 % singular values to capture 90 % variance


In [13]:
#так как именно s = 1, fraction = 5% будет использовано в LENS разложении, выведем количество сингулярных значений объясняющих 90% информации 
num_sv

134

Чем вводится больше аномалий, или чем они больше по размеру, тем больший процент сингулярных чисел необходим для объяснения 90% вариативности данных.

### LENS Decomposition

In [13]:
def soft_thresh(x, T):
    """Soft threshold function"""
    return  np.multiply(np.sign(x), np.maximum(np.abs(x) - T, 0))

In [14]:
def ADMM(D, frac_anomaly, frac_missing, random_indx_y10_miss, random_indx_x10_miss, m=182, n=3127):

    X = np.random.randint(-70, 61, size=(n, n)) 
    Y = np.random.randint(-70, 61, size=(m, n))
    Z = np.random.randint(-70, 61, size=(m, n))   
    A = np.array([np.where(x == None, 0, 1) for x in D])
    B = np.eye(182)
    C = np.eye(182)
    X_0 = np.random.randint(-70, 61, size=(n, n))
    X_1 = np.random.randint(-70, 61, size=(n, n))
    Y_0 = np.random.randint(-70, 61, size=(m, n))
    M_0 = np.random.randint(-1, 1, size=(n, n))
    M_1 = np.random.randint(-1, 1, size=(n, n))
    N = np.random.randint(-1, 1, size=(m, n))
    M = np.random.randint(-1, 1, size=(m, n))
    W = np.random.randint(-1, 1, size=(m, n))
    
    #списки индексов ячеек с  не пропущенными значениями
    non_elem = np.argwhere(D != None)
    D[np.isnan(D)] = 0
    E = np.array([[1 for col in range(3127)] for row in range(182)])
    for i in range(len(non_elem)):
        E[non_elem[i][0]][non_elem[i][1]] = 0
        W[non_elem[i][0]][non_elem[i][1]] = 0
    shape_d = (m, n)    #(182, 3127)
    shape_X = shape_Y = shape_d
    size_d = m*n
    #nu_D - это доля записей в D, которые не являются ни пропущенными, ни ошибочными.
    nu_D = 1 - (frac_missing+frac_anomaly)/size_d

    mu = 1.01
    p = 1.05
    
    J = D - A@X_0 - B@Y_0 - W
    alpha = (shape_X[0]**(0.5) + shape_X[1]**(0.5))*nu_D
    beta = (2*math.log(shape_Y[0]*shape_Y[1]))**(0.5)
    gamma = 1
    theta = 10
    K = 1
    P_1 = np.eye(n)
    q1_list_1 = [1] +  [0] * (n-1)
    q1_list_2 = [1, -1] + [0] * (n-2)
    Q_1 =  scipy.linalg.toeplitz(q1_list_1, q1_list_2)
    R_1 = np.zeros(n)

    #sigma_D определяется на каждой иттерации ADM алгоритма
    #как стандартное отклонение ряда значений J[i,j], где E[i,j]=0
    tmp_nonzero = []
    indices = np.nonzero(~np.isnan(E))
    non_elem = np.column_stack(indices)
    for i in range(len(non_elem)):
        tmp_nonzero.append(J[non_elem[i][0]][non_elem[i][1]]) 
    sigma_D = statistics.stdev(tmp_nonzero)
    sigma = theta*sigma_D
    MAX_ITER = 140
    for i in range(MAX_ITER):
        print("Iteration", i)

        #first step X
        J = 1/(K+1)*(X_1 + M_1/mu + X_0 + M_0/mu)
        t = alpha/(mu*(K+1))        
        # QR факторизация
        Q, R = np.linalg.qr(J)   
        u, s, vt = np.linalg.svd(J, full_matrices=False)
        S = np.diag(s)
        s = soft_thresh(S, t)
        s = np.diag(s)
        X = Q.dot(u[:,:2]).dot(np.diag(s[:2])).dot(vt[:2,:])

        #second step 
        J = X - M_1/mu
        R = P_1.transpose() @ R_1 @ Q_1 + ((mu*sigma)/gamma) * J
        up, sp, vtp = np.linalg.svd(P_1 @ P_1.transpose())
        uq, sq, vtq = np.linalg.svd(Q_1 @ Q_1.transpose())
        X_1 = up @ np.divide((vtp @ R @ uq), (sp*sq.transpose() + (mu*sigma/gamma))) @ vtq
        
        #third step with X0
        J_0 = X-M_0/mu
        J = D-np.matmul(B, Y_0)-np.matmul(C, Z)-W + M/mu
        X_0 = np.linalg.inv(A.transpose()@A + np.eye(n))@(A.transpose()@J + J_0)
        
        #fourth step Y
        J = Y_0 + N/mu
        t = beta/mu
        Y = soft_thresh(J, t)

        #fifth step Y_0
        J_0 = Y - N/mu
        J = D-np.matmul(A, X_0)-np.matmul(C, Z)-W + M/mu
        Y_0 = np.linalg.inv(B.transpose()@B + np.eye(m))@(B.transpose()@J + J_0)
        
        #sixth step Z
        J = D-np.matmul(A, X_0)-np.matmul(B, Y_0)-W + M/mu
        Z = np.linalg.inv(1/(mu*sigma)*np.eye(m) + np.matmul(C.transpose(), C))@(np.matmul(C.transpose(),J))
        
        #seventh step W
        W = np.multiply(E, (D-np.matmul(A, X_0)-np.matmul(B, Y_0)-np.matmul(C, Z) + M/mu))
        
        #eighth step sigma
        #sigma определяется на каждой иттерации ADM алгоритма, 
        #как стандартное отклонение ряда значений J[i,j], где E[i,j]=0
        theta = 10
        J = D-np.matmul(A, X_0)-np.matmul(B, Y_0)-W
        tmp_nonzero = []
        indices = np.nonzero(~np.isnan(E))
        non_elem = np.column_stack(indices)
        for i in range(len(non_elem)):
            tmp_nonzero.append(J[non_elem[i][0]][non_elem[i][1]])      
        
        sigma_D = statistics.stdev(tmp_nonzero)           
        sigma = theta*sigma_D

        #ninth step
        #Every itteration update M, M_0, N
        M = M + mu * (D-np.matmul(A, X_0)-np.matmul(B, Y_0)-np.matmul(C, Z)-W)
        M_0 = M_0 + mu * (X_0-X)
        M_1 = M_1 + mu*(X_1-X)
        N = N +  mu *  (Y_0-Y)
        
        #tenth step
        #Every itteration update mu
        mu = p*mu
        #Every 100 iterations, we multiply ρ by 1.05
        if isinstance(i//100, int):
            p_ = 1.05
            p = p*p_
        if sigma_D == 0:
            return D
        D = A @ X_0 + B @ Y_0 + C @ Z + W
    D = A @ X_0 + B @ Y_0 + C @ Z + W    
    return D


In [ ]:
%%time
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import TruncatedSVD
from sklearn.impute import KNNImputer
from scipy.sparse.linalg import svds
from sklearn.neighbors import KNeighborsRegressor

df_ = df.copy(deep=True)
D = np.asmatrix(df_)
tmp_arr = copy.deepcopy(main_arr)
frac_anomaly = round(182*3127*0.1)//2
#frac_anomaly = 0

new_loss_result_list = []
new_loss_result_list_2 = []
new_loss_result_list_3 = []

p_ = [0.2, 0.4, 0.8]
anomaly_size = [0.5, 1, 2, 5]
anomaly_ratio = [0, 0.05, 0.1, 0.2]
loss_probability = [0.2, 0.4, 0.8]
loss = 0.2
anomaly_r = 0.1
anomaly_s = 1

#for p in p_:
for anomaly_r in anomaly_ratio:
#for anomaly_s in anomaly_size:
#for loss in loss_probability:
    new_nmae_err_list = []
    new_nmae_err_list_2 = []
    new_nmae_err_list_3 = []
    nmae_err_list = []
    nmae_err_list_2 = []
    nmae_err_list_3 = []
    frac_missing = round(182*3127*loss)
    for i in range(1):
        #first technique
        random_indx_y10_miss = np.random.randint(0, 182, size=round(182*3127*loss))
        random_indx_x10_miss = np.random.randint(0, 3127, size=round(182*3127*loss))
        
#         #second technique
#         random_indx_y10_miss = np.array([i for i in range(182)])
#         random_indx_x10_miss = np.random.randint(0, 3127, size=round(182*3127*loss))
        
#         #third technique
#         random_indx_y10_miss = np.random.randint(0, 182, size=round(182*3127*loss))
#         random_indx_x10_miss = np.array([i for i in range(3127)])
        
#         #third technique
#         random_indx_y10_miss = np.random.randint(0, 100, size=round(100*150*loss))
#         random_indx_x10_miss = np.random.randint(0, 150, size=round(100*150*p))

#         #fourth technique
#         random_indx_y10_miss = np.array([i for i in range(100)])
#         random_indx_x10_miss = np.random.randint(0, 150, size=round(100*150*loss))
        
#         #fifth technique
#         random_indx_y10_miss = np.random.randint(0, 100, size=round(100*150*loss))
#         random_indx_x10_miss = np.array([i for i in range(150)])

        list_D = []
        list_result_D = []
        list_result_D_2 = []
        list_result_D_3 = []
        tmp_arr = copy.deepcopy(main_arr)
        for i in range(len(random_indx_y10_miss)):
            y = int(random_indx_y10_miss[i])
            x = int(random_indx_x10_miss[i])
            list_D.append(main_arr[y][x])
            tmp_arr[y][x] = None
        
        mean = np.nanmean(tmp_arr)
        tmp_arr_for_svd = copy.deepcopy(tmp_arr)
        tmp_arr_for_svd[np.isnan(tmp_arr_for_svd)] = 0
        
        indices = np.nonzero(np.isnan(tmp_arr_for_svd)) 
        random_indx_y10_anomal = np.random.randint(0, 182, size=round(182*3127*0.1))
        random_indx_x10_anomal = np.random.randint(0, 3127, size=round(182*3127*0.1))
    
        for i in range(len(random_indx_y10_anomal)//2):
            y = int(random_indx_y10_anomal[i])
            x = int(random_indx_x10_anomal[i])
            tmp_arr[y][x] += s_1
            tmp_arr_for_svd[y][x] += s_1
        
        U, sigma, V = np.linalg.svd(tmp_arr_for_svd)
        # Выберем только первые две компоненты для аппроксимации
        U = U[:, :2]
        sigma = np.diag(sigma[:2])
        V = V[:2, :]
        # Вычислим аппроксимированную матрицу
        D_approx = U @ sigma @ V

        # Применяем K ближайших соседей для локальной интерполяции:
        knn = KNeighborsRegressor(n_neighbors=2)
        matrix_interp = knn.fit(D_approx, main_arr).predict(D_approx)

        # Получаем итоговую матрицу, заменяя пропущенные значения после локальной интерполяции:
        D_svd_knn = copy.deepcopy(tmp_arr)
        D_svd_knn[np.isnan(D_svd_knn)] = matrix_interp[np.isnan(D_svd_knn)]
        
        for i in range(len(random_indx_y10_miss)):
            y = int(random_indx_y10_miss[i])
            x = int(random_indx_x10_miss[i])
            list_result_D_3.append(D_svd_knn[y][x])
            list_result_D_2.append(D_approx[y][x])
           
            
        m=182
        n=3127
        result_D = ADMM(tmp_arr, frac_anomaly, frac_missing, random_indx_y10_miss, random_indx_x10_miss, m, n)
        for i in range(len(random_indx_y10_miss)):
            y = int(random_indx_y10_miss[i])
            x = int(random_indx_x10_miss[i])
            list_result_D.append(result_D[y][x])         
        
        raznost = [x - y for x, y in zip(list_D, list_result_D)]
        raznost_2 = [x - y for x, y in zip(list_D, list_result_D_2)]
        raznost_3 = [x - y for x, y in zip(list_D, list_result_D_3)]
        new_nmae_err_list.append(np.mean(np.abs(raznost) / np.nanmean(np.abs(list_D))))
        new_nmae_err_list_2.append(np.mean(np.abs(raznost_2) / np.nanmean(np.abs(list_D))))
        new_nmae_err_list_3.append(np.mean(np.abs(raznost_3) / np.nanmean(np.abs(list_D))))
    
    new_loss_result_list.append(np.mean(new_nmae_err_list))
    new_loss_result_list_2.append(np.mean(new_nmae_err_list_2))
    new_loss_result_list_3.append(np.mean(new_nmae_err_list_3))   

print(new_loss_result_list)
print(new_loss_result_list_2)
print(new_loss_result_list_3)        